# Purposes

This jupyter notebook is to scrape the monthly visa statistics data from the U.S. Department of State.

This notebook perform two tasks:

1. Create a catalog dataframe that includes the URL and timestamp of all webpages to be scraped.
2. Batch download online .pdf documents to local drive.


This jupyter notebook was created on 05/29/2023, Monday, 17:23 PM

In [1]:
import time as tm
start_time = tm.strftime("%m/%d/%Y, %A, %H:%M %p")
print("This jupyter notebook was created on", start_time)

This jupyter notebook was created on 05/29/2023, Monday, 17:23 PM


In [1]:
import time as tm
import requests
from scrapy import Selector
import re
import datetime as dt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd

# URLs

The main tasks in this step are:
- Capture the unique patterns in the urls that I'd like to scrape
- Make sure all urls are included. If the pattern doesn't cover all urls, append them manually to the list.

## Main URL

In [2]:
main_url = 'https://travel.state.gov/content/travel/en/legal/visa-law0/visa-statistics/immigrant-visa-statistics/monthly-immigrant-visa-issuances.html'

In [3]:
main_html = requests.get(main_url).content
main_selector = Selector(text = main_html)
main_selector

<Selector xpath=None data='<html><head><script type="text/javasc...'>

In [4]:
type(main_selector) # This object has no length

scrapy.selector.unified.Selector

## Developer Tools in Browser

Open the setting of a browser, choose developer tools, and check the elements:

- Copy Xpath: `/html/body/div[3]/div[7]/div[2]/div[1]/div[2]/div[2]/div/ul[7]/li[11]/a[1]`
- Copy element: `<a href="/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202023%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf" target="_blank" adhocenable="false" class="pdf_link">March 2023 - IV Issuances by FSC or Place of Birth and Visa Class<span class="far fa-file-pdf pdf_link"></span></a>`
- Copy Selector: `body > div.tsg-rwd-body-frame-row > div.contentbody > div.tsg-rwd-main-copy-frame > div.tsg-rwd-main-copy-body-frame.withrail > div.tsg-rwd-content-page-parsysxxx.parsys > div:nth-child(2) > div > ul:nth-child(15) > li:nth-child(11) > a.pdf_link`

In [5]:
# contains(@attri-name, "string-expr")
all_links = main_selector.xpath('//*[contains(@href,"Class.pdf")]/@href').extract()

In [6]:
all_links[:5]

['/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/March%202017%20-%20IV%20Issuances%20by%20Post%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/APRIL%202017%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/APRIL%202017%20-%20IV%20Issuances%20by%20Post%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MAY%202017%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MAY%202017%20-%20IV%20Issuances%20by%20Post%20and%20Visa%20Class.pdf']

In [7]:
fsc_links = [link for link in all_links if "FSC" in link]
len(fsc_links)

73

## Append additional URLs

Although the URLs from a webpage usually have patterns, the pattern is usually not guaranteed. If all urls we want cannot be in(ex)cluded by specific patterns, copy the urls from the webpage and append them.

- Element: `<a href="/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/March%202017%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class%20-%20Worldwide.pdf" target="_blank" adhocenable="false" class="pdf_link">March 2017 - IV Issuances by FSC or Place of Birth and Visa Class<span class="far fa-file-pdf pdf_link"></span></a>`
- Xpath: `/html/body/div[3]/div[7]/div[2]/div[1]/div[2]/div[2]/div/ul[1]/li[1]/a`
- Full Xpath: `/html/body/div[3]/div[7]/div[2]/div[1]/div[2]/div[2]/div/ul[1]/li[1]/a`

In [8]:
fsc_links.append('/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/March%202017%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class%20-%20Worldwide.pdf')
print(len(fsc_links))
fsc_links = list(set(fsc_links))
print(len(fsc_links))

74
74


## Quality Check for URLs

In [9]:
[fsc for fsc in fsc_links if ('March' in fsc or 'MARCH' in fsc)]

['/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202023%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202021%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202018%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202019%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202020%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202022%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 '/content/dam/visas/Statistics/Immigrant-Statistics/Month

In [10]:
#years = ['2017','2018','2019','2020','2021','2022','2023']
# number of links in each year, from March 2017 (10 months in 2017) until March 2023
print("2017:",len([link for link in fsc_links if "2017" in link]))
print("2018:",len([link for link in fsc_links if "2018" in link]))
print("2019:",len([link for link in fsc_links if "2019" in link]))
print("2020:",len([link for link in fsc_links if "202020" in link])) # not a typo but by the actual links
print("2021:",len([link for link in fsc_links if "2021" in link]))
print("2022:",len([link for link in fsc_links if "2022" in link]))
print("2023:",len([link for link in fsc_links if "2023" in link]))

2017: 10
2018: 12
2019: 12
2020: 12
2021: 12
2022: 12
2023: 4


## Full URLs

In [11]:
prefix = "https://travel.state.gov"

In [12]:
# Concatenate the prefix with every link
for i, link in enumerate(fsc_links):
    if link.startswith('/content'):
        fsc_links[i] = prefix + link
fsc_links[:5]      

['https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/JULY2021%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 'https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/JUNE%202017%20-%20IV%20Issuances%20by%20FSC%20and%20Visa%20Class.pdf',
 'https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/OCTOBER2021%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 'https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/AUGUST%202022%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf',
 'https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/MARCH%202023%20-%20IV%20Issuances%20by%20FSC%20or%20Place%20of%20Birth%20and%20Visa%20Class.pdf']

In [13]:
# save all the FSC URLs
file = open('URL_visa_issuance_FSC.txt','w')
for url in fsc_links:
    file.write(url + "\n")
file.close()    

# Build Catalog Info

## Split URLs by Observed Patterns

In [15]:
fsc_links[5].split('/')[-1].split('%')

['NOVEMBER',
 '202022',
 '20-',
 '20IV',
 '20Issuances',
 '20by',
 '20FSC',
 '20or',
 '20Place',
 '20of',
 '20Birth',
 '20and',
 '20Visa',
 '20Class.pdf']

In [16]:
month = [None]*len(fsc_links)
year = [None]*len(fsc_links)
for i, link in enumerate(fsc_links):
    month[i] = link.split('/')[-1].split('%')[0].upper()
    year[i] = link.split('/')[-1].split('%')[1][2:]

## Data Cleaning

In [17]:
print(set(month))
print(set(year))

{'JANUARY', 'MAY', 'JUNE2021', 'OCTOBER2021', 'APRIL', 'AUGUST', 'JANUARY2022', 'SEPT', 'SEPTEMBER', 'NOVEMBER2021', 'MARCH', 'AUGUST2021', 'DECEMBER2021', 'NOVEMBER', 'JUNE', 'DECEMBER', 'FEBRUARY', 'JULY2021', 'MAY2021', 'OCTOBER', 'JULY', 'SEPTEMBER2021'}
{'2022', '2023', '2019', '2017', '2018', '-', '2021', '2020'}


Unfortunately but not surprisingly, the URLs are not uniformly structured. When I extract the month and year by their location in a URL from observed patterns, there're exceptions. For example, 'JUNE2021', 'MAY2021', and 'DECEMBER2021', as shown in the month list, should be further split. Also, in the list of year, there is the '-'. 

Solutions to these problems should be data-dependent. Here, I try to get the index for the problematic elements in the year list.

In [18]:
temp_list = []
for i,yr in enumerate(year):
    if len(yr)!=4:
        temp_list.append(i)
temp_list        

[0, 2, 9, 17, 27, 32, 34, 44, 65]

In [19]:
FSC_catalog = pd.DataFrame({
    'url': fsc_links,
    'year': year,
    'month': month
})

In [20]:
FSC_catalog.iloc[temp_list]

,url,year,month
0,https://travel.state.gov/content/dam/visas/Sta...,-,JULY2021
2,https://travel.state.gov/content/dam/visas/Sta...,-,OCTOBER2021
9,https://travel.state.gov/content/dam/visas/Sta...,-,SEPTEMBER2021
17,https://travel.state.gov/content/dam/visas/Sta...,-,JUNE2021
27,https://travel.state.gov/content/dam/visas/Sta...,-,MAY2021
32,https://travel.state.gov/content/dam/visas/Sta...,-,JANUARY2022
34,https://travel.state.gov/content/dam/visas/Sta...,-,DECEMBER2021
44,https://travel.state.gov/content/dam/visas/Sta...,-,AUGUST2021
65,https://travel.state.gov/content/dam/visas/Sta...,-,NOVEMBER2021


In [21]:
# repeat the creation of the catalog dataframe in the same chunk of these ad-hoc editions
# this way, we get the finished catalog dataframe within one chunk
FSC_catalog = pd.DataFrame({
    'url': fsc_links,
    'year': year,
    'month': month
})
# call `.str` method and slice from the other column
FSC_catalog['year'].iloc[temp_list] = FSC_catalog['month'].iloc[temp_list].str[-4:]
# then, remove the year information from the month column
FSC_catalog['month'].iloc[temp_list] = FSC_catalog['month'].iloc[temp_list].str[:-4]
# fix the irregular spelling of 'SEPT' for 'SEPTEMBER'
FSC_catalog['month'].iloc[FSC_catalog[FSC_catalog['month'] == 'SEPT'].index] = 'SEPTEMBER'
# confirm the month variable only includes the 12 months
print(set(FSC_catalog['month']))

{'FEBRUARY', 'JANUARY', 'MAY', 'OCTOBER', 'SEPTEMBER', 'APRIL', 'JULY', 'MARCH', 'AUGUST', 'JUNE', 'NOVEMBER', 'DECEMBER'}


In [22]:
# fixed!
print(FSC_catalog.shape)
FSC_catalog.iloc[temp_list]

(74, 3)


,url,year,month
0,https://travel.state.gov/content/dam/visas/Sta...,2021,JULY
2,https://travel.state.gov/content/dam/visas/Sta...,2021,OCTOBER
9,https://travel.state.gov/content/dam/visas/Sta...,2021,SEPTEMBER
17,https://travel.state.gov/content/dam/visas/Sta...,2021,JUNE
27,https://travel.state.gov/content/dam/visas/Sta...,2021,MAY
32,https://travel.state.gov/content/dam/visas/Sta...,2022,JANUARY
34,https://travel.state.gov/content/dam/visas/Sta...,2021,DECEMBER
44,https://travel.state.gov/content/dam/visas/Sta...,2021,AUGUST
65,https://travel.state.gov/content/dam/visas/Sta...,2021,NOVEMBER


## Create Timestamp `pd.to_datetime() + MonthEnd(0)`

In [23]:
# concatenate the year and month data
FSC_catalog['mmyy'] = FSC_catalog['year'].str.cat(FSC_catalog['month'], sep='-')

In [24]:
# https://stackoverflow.com/questions/37354105/find-the-end-of-the-month-of-a-pandas-dataframe-series
from pandas.tseries.offsets import MonthEnd
# because the visa statistics is the summary as of the end of the month, I set the date to the last date
# use `+ MonthEnd(0)`
FSC_catalog['mmyy'] =\
pd.to_datetime(FSC_catalog['mmyy']\
               , infer_datetime_format=True, errors = 'ignore')\
+ MonthEnd(0)
FSC_catalog = FSC_catalog.sort_values('mmyy', ascending=True).reset_index(drop=True)
FSC_catalog

,url,year,month,mmyy
0,https://travel.state.gov/content/dam/visas/Sta...,2017,MARCH,2017-03-31
1,https://travel.state.gov/content/dam/visas/Sta...,2017,APRIL,2017-04-30
2,https://travel.state.gov/content/dam/visas/Sta...,2017,MAY,2017-05-31
3,https://travel.state.gov/content/dam/visas/Sta...,2017,JUNE,2017-06-30
4,https://travel.state.gov/content/dam/visas/Sta...,2017,JULY,2017-07-31
...,...,...,...,...
69,https://travel.state.gov/content/dam/visas/Sta...,2022,DECEMBER,2022-12-31
70,https://travel.state.gov/content/dam/visas/Sta...,2023,JANUARY,2023-01-31
71,https://travel.state.gov/content/dam/visas/Sta...,2023,FEBRUARY,2023-02-28
72,https://travel.state.gov/content/dam/visas/Sta...,2023,MARCH,2023-03-31


In [25]:
import time as tm
# output the catalog data
FSC_catalog.to_csv("FSC_catalog.csv", index=False)
print("The catalog data was last saved on",tm.strftime("%m/%d/%Y, %A, %H:%M %p"))

The catalog data was last saved on 06/01/2023, Thursday, 22:47 PM


# Download ALL .pdf Documents

Python only needs the full URLs of online .pdf documents in order to save them to local drive.

In [57]:
import os
from urllib import request

In [56]:
%%time
for i in range(len(FSC_catalog)):
    fullfilename = os.getcwd() + '/iv/'+ 'iv_'+str(FSC_catalog['mmyy'][i])[:10] + '.pdf'
    request.urlretrieve(FSC_catalog['url'][i], fullfilename)

CPU times: user 2.55 s, sys: 516 ms, total: 3.07 s
Wall time: 49 s


In [59]:
print("We have just downloaded",len(os.listdir('iv')), ".pdf files via web-scraping.\n")
print(os.listdir('iv')[:20])

We have just downloaded 74 .pdf files via web-scraping.

['iv_2020-07-31.pdf', 'iv_2020-02-29.pdf', 'iv_2019-01-31.pdf', 'iv_2018-02-28.pdf', 'iv_2017-06-30.pdf', 'iv_2018-07-31.pdf', 'iv_2021-01-31.pdf', 'iv_2022-06-30.pdf', 'iv_2022-08-31.pdf', 'iv_2021-11-30.pdf', 'iv_2017-04-30.pdf', 'iv_2018-09-30.pdf', 'iv_2022-04-30.pdf', 'iv_2021-03-31.pdf', 'iv_2017-08-31.pdf', 'iv_2018-05-31.pdf', 'iv_2019-03-31.pdf', 'iv_2020-05-31.pdf', 'iv_2019-11-30.pdf', 'iv_2020-09-30.pdf']


In [60]:
print("This jupyter notebook was last completed at", tm.strftime("%m/%d/%Y, %A, %H:%M %p"))

This jupyter notebook was last completed at 06/02/2023, Friday, 00:07 AM
